task name: Resolve TextTransform Task
task description: Implement the TextTransform example using C# code

In [ ]:
#r "nuget:Microsoft.ML"


In [ ]:
using System;
using System.Collections.Generic;
using Microsoft.ML;
using Microsoft.ML.Data;
using Microsoft.ML.Transforms.Text;

In [ ]:
var ml = new MLContext();

In [ ]:
public class SampleSentimentData
{
    public bool Sentiment { get; set; }
    public string SentimentText { get; set; }
}

var data = new List<SampleSentimentData>() {
            new SampleSentimentData { Sentiment = true,
                SentimentText = "Best game I've ever played." },

            new SampleSentimentData { Sentiment = false,
                SentimentText = "==RUDE== Dude, 2" },

            new SampleSentimentData { Sentiment = true,
                SentimentText = "Until the next game," +
                "this is the best Xbox game!" } };

var trainData = ml.Data.LoadFromEnumerable(data);

In [ ]:
string defaultColumnName = "DefaultTextFeatures";
var default_pipeline = ml.Transforms.Text
            .FeaturizeText(defaultColumnName, "SentimentText");

string customizedColumnName = "CustomizedTextFeatures";
var customized_pipeline = ml.Transforms.Text
            .FeaturizeText(customizedColumnName,
            new TextFeaturizingEstimator.Options

            {
                KeepPunctuations = false,
                KeepNumbers = false,
                OutputTokensColumnName = "OutputTokens",
                StopWordsRemoverOptions =
                new StopWordsRemovingEstimator.Options()
                {
                    Language = TextFeaturizingEstimator.Language.English
                },

            }, "SentimentText");

var transformedData_default = default_pipeline.Fit(trainData)
            .Transform(trainData);

var transformedData_customized = customized_pipeline.Fit(trainData)
            .Transform(trainData);

In [ ]:
Action<string, IEnumerable<VBuffer<float>>> printHelper = (columnName,
            column) =>

    {
        Console.WriteLine(
            $"{columnName} column obtained post-transformation.");

        foreach (var featureRow in column)
        {
            foreach (var value in featureRow.GetValues())
                Console.Write($"{value} ");
            Console.WriteLine("");
        }

        Console.WriteLine(
            "===================================================");

    };

var defaultColumn = transformedData_default
            .GetColumn<VBuffer<float>>(transformedData_default
            .Schema[defaultColumnName]);

printHelper(defaultColumnName, defaultColumn);

var customizedColumn = transformedData_customized
            .GetColumn<VBuffer<float>>(transformedData_customized
            .Schema[customizedColumnName]);

printHelper(customizedColumnName, customizedColumn);